# YiJing Modul Code Review Notebook

Dieses Notebook dient zur systematischen Code-Review und zum Testen des YiJing-Moduls.

## Bereiche der Review:
1. Grundlegende Funktionalität
2. Fehlerbehandlung
3. Async Support
4. Performance
5. Ressourcenverwaltung

In [1]:
# Imports und Setup
import sys
from pathlib import Path
import logging
from pprint import pprint

# Logging Setup
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Projektpfad zum Python-Path hinzufügen
project_dir = Path.cwd()  # Da das Notebook im Root liegt
if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))

## 1. Grundlegende Funktionalität

Zuerst testen wir die Kernfunktionalität des Moduls.

In [2]:
import ollama
from IPython.display import display, Markdown

# Auswahl des Modells
model_speed = {
    'dumb': 'llama3.1:latest',
    'vision': 'llama3.2-vision:latest',
    'fast': 'llama3.2:1b:latest',
    'clever': 'llama3.2:latest',
    'assistent': 'dolphin-mistral:latest'
    }

#selected_model = model_speed['dumb']
#selected_model = model_speed['vision']
#selected_model = model_speed['fast']
selected_model = model_speed['clever']
#selected_model = model_speed['uncensored']

print("Model:", selected_model)

2024-12-31 09:36:27,794 - httpx - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-12-31 09:36:27,796 - httpx - DEBUG - load_verify_locations cafile='/Users/johanneskaindl/miniconda3/envs/gemini_310/lib/python3.10/site-packages/certifi/cacert.pem'


Model: llama3.2:latest


In [3]:
# Import der Hauptkomponenten
from yijing import (
    YijingOracle,
    cast_hypergram,
    generiere_erweiterte_weissagung,
    formatiere_weissagung_markdown
)
from yijing.config import ModelType

In [4]:
# Test der verschiedenen Modelle
models = {
    'genai': {
        'type': ModelType.GENAI,
        'model': 'models/gemini-1.5-flash'
    },
    'ollama': {
        'type': ModelType.OLLAMA,
        'model': selected_model
    }
}

# Wähle ein Modell für den Test
selected_model = models['ollama']  # oder 'genai'

custom_settings = {
    "model_type": selected_model['type'],
    "active_model": selected_model['model'],
    "debug": True
}

oracle = YijingOracle(custom_settings=custom_settings)

2024-12-31 09:36:40,016 - yijing.core.manager - DEBUG - Initialized HexagramManager with resources path: /Users/johanneskaindl/Documents/0_inbox/2_projekte/1_coding/YijingOracle/yijing/resources


In [9]:
# Test einer Weissagung
test_frage = "Was sagt das I Ging über meine aktuelle Situation?"
weissagung = oracle.get_response(test_frage)

# Ausgabe der Antwort
print("Strukturierte Antwort:")
pprint(weissagung)

2024-12-31 09:06:49,610 - yijing.core.oracle - INFO - Verarbeite Frage im single Modus mit Modell ollama
2024-12-31 09:06:49,610 - yijing.core.oracle - INFO - Verarbeite Frage im single Modus mit Modell ollama
2024-12-31 09:06:49,612 - yijing.core.generator - DEBUG - Generating six random lines for hypergram
2024-12-31 09:06:49,613 - yijing.core.generator - DEBUG - Generated line values: [7, 6, 6, 6, 9, 9]
2024-12-31 09:06:49,613 - yijing.models.hexagrams - DEBUG - Validating hypergram lines
2024-12-31 09:06:49,614 - yijing.models.hexagrams - DEBUG - Finding changing lines
2024-12-31 09:06:49,614 - yijing.models.lines - DEBUG - Checking if line 7 is changing
2024-12-31 09:06:49,615 - yijing.models.lines - DEBUG - Checking if line 6 is changing
2024-12-31 09:06:49,616 - yijing.models.lines - DEBUG - Checking if line 6 is changing
2024-12-31 09:06:49,617 - yijing.models.lines - DEBUG - Checking if line 6 is changing
2024-12-31 09:06:49,618 - yijing.models.lines - DEBUG - Checking if line

Strukturierte Antwort:
{'answer': 'Vielen Dank für die Gelegenheit, dir als weises I Ging Orakel zu '
           'helfen!\n'
           '\n'
           'Basierend auf deiner Frage und den hexagrammen, die du '
           'bereitgestellt hast, sehe ich folgende Schlüsselaspekte in deiner '
           'aktuellen Situation:\n'
           '\n'
           '**Die Wendezeit ist vorbei**: Das ausgangshexagramm FU / DIE '
           'WIEDERKEHRT (DIE WENDEZEIT) deutet darauf hin, dass die dunklen '
           'Linien die lichten alle nach oben hinausgedrängt haben und nun '
           'wieder ein lichter Strich von unten her in das Zeichen eintritt. '
           'Das bedeutet, dass die Zeit des Dunkels vorüber ist und eine neue '
           'Zeit der Wende beginnt.\n'
           '\n'
           '**Die Wiederkehr**: Der resultierende hexagramm DSING / DER '
           'BRUNNEN zeigt an, dass es eine Zeit der Erneuerung und des '
           'Umkehrens gibt. Die Wiederkehr kann als eine Gelegenhei

In [10]:
from jupyter_json_viewer import display_json

In [12]:
display_json(weissagung,
             dark_mode=True,
             )

## 2. Fehlerbehandlung

Testen der Fehlerbehandlung in verschiedenen Szenarien.

In [5]:
from pydantic import ValidationError
from yijing.enums import ModelType

In [16]:
def test_error_cases():
    """Test verschiedener Fehlerfälle für das YijingOracle"""
    
    # Test 1: Ungültige Konfiguration
    try:
        invalid_settings = {
            "model_type": "invalid_type",
            "active_model": ""
        }
        oracle_invalid = YijingOracle(custom_settings=invalid_settings)
        assert False, "Sollte ValidationError auslösen"
    except ValidationError as e:
        print(f"✓ Erwarteter ValidationError bei ungültiger Konfiguration: {e}\n")
    
    # Test 2: Leere Frage
    oracle = YijingOracle(custom_settings={
        "model_type": ModelType.OLLAMA,
        "active_model": "llama3.2:latest"
    })
    try:
        weissagung = oracle.get_response("")
        assert False, "Sollte ValueError auslösen"
    except ValueError as e:
        print(f"✓ Erwarteter ValueError bei leerer Frage: {e}\n")

test_error_cases()

2024-12-31 09:29:16,750 - yijing.core.manager - DEBUG - Initialized HexagramManager with resources path: /Users/johanneskaindl/Documents/0_inbox/2_projekte/1_coding/YijingOracle/yijing/resources
2024-12-31 09:29:16,751 - yijing.core.oracle - INFO - Verarbeite Frage im single Modus mit Modell ollama
2024-12-31 09:29:16,751 - yijing.core.oracle - INFO - Verarbeite Frage im single Modus mit Modell ollama
2024-12-31 09:29:16,752 - yijing.core.generator - DEBUG - Generating six random lines for hypergram
2024-12-31 09:29:16,752 - yijing.core.generator - DEBUG - Generated line values: [9, 8, 9, 8, 7, 8]
2024-12-31 09:29:16,753 - yijing.models.hexagrams - DEBUG - Validating hypergram lines
2024-12-31 09:29:16,753 - yijing.models.hexagrams - DEBUG - Finding changing lines
2024-12-31 09:29:16,753 - yijing.models.lines - DEBUG - Checking if line 9 is changing
2024-12-31 09:29:16,754 - yijing.models.lines - DEBUG - Checking if line 8 is changing
2024-12-31 09:29:16,754 - yijing.models.lines - DEB

✓ Erwarteter ValidationError bei ungültiger Konfiguration: 1 validation error for Settings
model_type
  Input should be 'genai' or 'ollama' [type=enum, input_value='invalid_type', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/enum



2024-12-31 09:30:33,760 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Tue, 31 Dec 2024 08:30:33 GMT'), (b'Transfer-Encoding', b'chunked')])
2024-12-31 09:30:33,762 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2024-12-31 09:30:33,763 - httpcore.http11 - DEBUG - receive_response_body.started request=<Request [b'POST']>
2024-12-31 09:30:33,764 - httpcore.http11 - DEBUG - receive_response_body.complete
2024-12-31 09:30:33,765 - httpcore.http11 - DEBUG - response_closed.started
2024-12-31 09:30:33,766 - httpcore.http11 - DEBUG - response_closed.complete


AssertionError: Sollte ValueError auslösen

## 3. Async Support

Test der asynchronen Funktionalität.

In [6]:
import asyncio

async def test_async_functionality():
    """Test der asynchronen Funktionen"""
    # Mehrere gleichzeitige Anfragen
    fragen = [
        "Was sagt das I Ging über meine Beziehungen?",
        "Wie entwickelt sich meine berufliche Situation?",
        "Was sollte ich in der aktuellen Situation beachten?"
    ]
    
    tasks = [oracle.get_response_async(frage) for frage in fragen]
    results = await asyncio.gather(*tasks)
    
    return results

# Ausführung des async Tests
async_results = await test_async_functionality()
print("Async Ergebnisse:")
for i, result in enumerate(async_results, 1):
    print(f"\nAntwort {i}:")
    print(result['answer'])

2024-12-31 09:36:54,098 - yijing.core.oracle - INFO - Starte asynchrone Verarbeitung im single Modus mit Modell ollama
2024-12-31 09:36:54,098 - yijing.core.oracle - INFO - Starte asynchrone Verarbeitung im single Modus mit Modell ollama
2024-12-31 09:36:54,099 - yijing.core.oracle - INFO - Starte asynchrone Verarbeitung im single Modus mit Modell ollama
2024-12-31 09:36:54,099 - yijing.core.oracle - INFO - Starte asynchrone Verarbeitung im single Modus mit Modell ollama
2024-12-31 09:36:54,100 - yijing.core.oracle - INFO - Starte asynchrone Verarbeitung im single Modus mit Modell ollama
2024-12-31 09:36:54,100 - yijing.core.oracle - INFO - Starte asynchrone Verarbeitung im single Modus mit Modell ollama
2024-12-31 09:36:54,105 - yijing.core.generator - DEBUG - Generating six random lines for hypergram
2024-12-31 09:36:54,106 - yijing.core.generator - DEBUG - Generating six random lines for hypergram
2024-12-31 09:36:54,106 - yijing.core.generator - DEBUG - Generated line values: [9, 9

ModelResponseError: Ungültige Antwort von Modell llama3.2:latest: 'Settings' object has no attribute 'OLLAMA_HOST'

2024-12-31 09:36:54,593 - yijing.core.manager - DEBUG - Generating consultation prompt for question: Wie entwickelt sich meine berufliche Situation?
2024-12-31 09:36:54,594 - yijing.core.manager - DEBUG - Generating consultation prompt for question: Was sagt das I Ging über meine Beziehungen?
2024-12-31 09:36:54,594 - yijing.core.oracle - ERROR - Unerwarteter Fehler bei der asynchronen Generierung der Antwort
Traceback (most recent call last):
  File "/Users/johanneskaindl/Documents/0_inbox/2_projekte/1_coding/YijingOracle/yijing/core/oracle.py", line 748, in get_response_async
    response_text = await self._get_ollama_response_async(prompt)
  File "/Users/johanneskaindl/Documents/0_inbox/2_projekte/1_coding/YijingOracle/yijing/core/oracle.py", line 580, in _get_ollama_response_async
    if not self.settings.OLLAMA_HOST:
  File "/Users/johanneskaindl/miniconda3/envs/gemini_310/lib/python3.10/site-packages/pydantic/main.py", line 856, in __getattr__
    raise AttributeError(f'{type(sel

## 4. Performance

Analyse der Performance-Metriken.

In [ ]:
import time
import statistics

def measure_performance(n_iterations=5):
    """Misst die Performance der Hauptfunktionen"""
    timings = {
        'hypergram_gen': [],
        'oracle_response': []
    }
    
    for _ in range(n_iterations):
        # Zeitmessung für Hypergram-Generierung
        start = time.time()
        hypergram = cast_hypergram()
        timings['hypergram_gen'].append(time.time() - start)
        
        # Zeitmessung für Oracle-Antwort
        start = time.time()
        response = oracle.get_response("Test Frage")
        timings['oracle_response'].append(time.time() - start)
    
    # Statistiken berechnen
    stats = {}
    for key, values in timings.items():
        stats[key] = {
            'mean': statistics.mean(values),
            'std': statistics.stdev(values),
            'min': min(values),
            'max': max(values)
        }
    
    return stats

performance_stats = measure_performance()
print("Performance Statistiken:")
pprint(performance_stats)

## 5. Ressourcenverwaltung

Überprüfung der Ressourcenverwaltung und des Speicherverbrauchs.

In [ ]:
import psutil
import os

def monitor_resources(func):
    """Überwacht Ressourcenverbrauch einer Funktion"""
    process = psutil.Process(os.getpid())
    
    # Speicherverbrauch vor Ausführung
    mem_before = process.memory_info().rss
    
    # Funktion ausführen
    result = func()
    
    # Speicherverbrauch nach Ausführung
    mem_after = process.memory_info().rss
    mem_diff = mem_after - mem_before
    
    print(f"Speicherverbrauch: {mem_diff / 1024 / 1024:.2f} MB")
    return result

# Test der Ressourcennutzung
def test_function():
    return oracle.get_response("Test Frage")

result = monitor_resources(test_function)

## Zusammenfassung

Basierend auf den Tests können wir folgende Aspekte analysieren:
1. Funktionalität und Korrektheit
2. Fehlerbehandlung und Robustheit
3. Performance und Skalierbarkeit
4. Ressourceneffizienz
5. Code-Qualität und Wartbarkeit